In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.special import j0, j1



factor_levels = {
    'Pressure': [0.02, 3.51, 7],
    'Youngs modulus': [2.1, 13.55, 25],
    'Poisson ratio': [0.2, 0.255, 0.31],
    'Depth': [800, 2025, 3250],
    'Radius': [400, 2750, 5100],
    'Thickness': [20, 348, 676]
}


file_path = 'Threelevels_DOE.xlsx'
df = pd.read_excel(file_path, 'Sheet1')


df_cleaned = df[['Pressure', 'Youngs modulus', 'Poisson ratio', 'Depth', 'Radius', 'Thickness']]


alpha_B = 1 


def get_factor_value(factor, level):
    return factor_levels[factor][level - 1]


def integrand(alpha, r, d, R):
    return np.exp(-alpha * d) * j1(alpha * R) * j0(alpha * r)


def uz_formula_corrected(r, Cm, h, dp_res, v, R, d):
    integral_value, _ = quad(integrand, 0, np.inf, args=(r, d, R), epsabs=1e-10, epsrel=1e-10)
    uz = 2 * Cm * (1 - v) * h * dp_res * R * integral_value
    return uz * 1000  


def compute_vertical_displacement(row):

    pressure_level = row['Pressure']
    youngs_modulus_level = row['Youngs modulus']
    poisson_ratio_level = row['Poisson ratio']
    depth_level = row['Depth']
    radius_level = row['Radius']
    thickness_level = row['Thickness']
    

    pressure = get_factor_value('Pressure', pressure_level) * 1e6  
    youngs_modulus = get_factor_value('Youngs modulus', youngs_modulus_level) * 1e9 
    poisson_ratio = get_factor_value('Poisson ratio', poisson_ratio_level)
    depth = get_factor_value('Depth', depth_level)
    radius = get_factor_value('Radius', radius_level)
    thickness = get_factor_value('Thickness', thickness_level)
    
    Cm = alpha_B * ((1 + poisson_ratio) * (1 - 2 * poisson_ratio)) / ((1 - poisson_ratio) * youngs_modulus)
    

    uz = uz_formula_corrected(0, Cm, thickness, pressure, poisson_ratio, radius, depth)  
    return uz


df_cleaned['Vertical Displacement (mm)'] = df_cleaned.apply(compute_vertical_displacement, axis=1)


output_file = 'r0_three_level.csv'
df_cleaned.to_csv(output_file, index=False)

print(f"Results have been saved to {output_file}")


Results have been saved to r0_three_level.csv


C:\Users\14185\AppData\Local\Temp\ipykernel_13692\3407481469.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Vertical Displacement (mm)'] = df_cleaned.apply(compute_vertical_displacement, axis=1)
